In [219]:
from bs4 import BeautifulSoup as bsoup
import os
import sys


In [220]:
def eachFile(filepath):
    """
    get all the files in the dir
    """
    pathDir =  os.listdir(filepath)
    files = []
    for allDir in pathDir:
        child = os.path.join('%s%s' % (filepath, allDir))
        files.append(child)
    return files

def get_text(path):
    """
    get the text value in the file
    """
    values = []
    text = open(path, 'r')
    for line in text:
        values.append(line)
    return ' '.join(values)

In [221]:
files = eachFile('./Desktop/sentiment/movie/')
movie_files = []
for file in files:
    if not file.find('.html') == -1:
        movie_files.append(file)

In [223]:
# soup = bsoup(get_text(movie_files[0]))
import nltk
movie_text = []
for file in movie_files:
    htmlfile = open(file, 'r',encoding='ISO-8859-1')
    try:
        htmlpage = htmlfile.read()
    except:
        print(file)
    soup = bsoup(htmlpage, "html.parser")
    if (soup.find('p') is None):
        movie_text.append('')
    else:
        sent = soup.find('p').string
#         tokens=nltk.word_tokenize(str(sent))
        movie_text.append(str(sent))
#print(movie_text[3])

In [224]:
import pandas as pd
import numpy as np
predict_data = pd.DataFrame(columns = ['file_name' , 'content'])
predict_data['file_name'] = movie_files
predict_data['content'] = movie_text

In [225]:
from nltk.stem.lancaster import LancasterStemmer
LancasterStem=LancasterStemmer()

In [226]:
# get the training data
files = eachFile('./Desktop/sentiment/review_polarity/txt_sentoken/neg/')
neg_files = []
for file in files:
    if not file.find('.txt') == -1:
        neg_files.append(file)

files = eachFile('./Desktop/sentiment/review_polarity/txt_sentoken/pos/')
pos_files = []
for file in files:
    if not file.find('.txt') == -1:
        pos_files.append(file)

neg_text = []
pos_text = []
for file in neg_files:
    text = get_text(file)
    neg_text.append(text)
for file in pos_files:
    text = get_text(file)
    pos_text.append(text)

In [227]:
# get the category of the word by using nltk method
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
tags = set(['NN', 'NNP', 'NNS', 'NNPS', 'VB', 'VBG', 'VBN', 'VBP', 'VBZ', 'RB', 'RBR', 'RBS', 'JJ', 'JJR', 'JJS'])
def filter(text):
    #using nltk to filter the stopword
    wnl = WordNetLemmatizer()
    sent = nltk.word_tokenize(text)
#    print (sent)
#    words = [LancasterStem.stem(w) for w in sent if w not in stopwords.words('english')]
    words=[]
    for w in sent: 
        if w not in stopwords.words('english'):
            words.append(w)
        
    #words = [for w in sent if w not in stopwords.words('english')]
    #print(words)
#     get the categoyr of word
    pos_tags =nltk.pos_tag(words)
    ret = []
    #filter
    for word,pos in pos_tags:
        
        new = wnl.lemmatize(word)
        
        if (pos in tags):
            ret.append(new)
#    print(ret)
    return ' '.join(ret)
#     return ' '.join(words)
    

In [228]:
#get the text after filtering
neg_txt = list(map(filter, neg_text))
pos_txt = list(map(filter, pos_text))

In [229]:
print(len(neg_text))

12500


In [230]:
print(len(pos_text))

12500


In [231]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
# structer the data
neg_pd = pd.DataFrame(columns = ['content', 'label'])
pos_pd = pd.DataFrame(columns = ['content', 'label'])
neg_pd['content'] = neg_txt
neg_pd['label'] = np.ones((12500,1), dtype = np.int)
pos_pd['content'] = pos_txt
pos_pd['label'] = np.zeros((12500,1), dtype = np.int)
data = pd.concat([pos_pd, neg_pd], axis = 0, ignore_index = True)
display(data.head(10))

,content,label
0,Bromwell High cartoon comedy time program scho...,0
1,Homelessness Houselessness George Carlin issue...,0
2,Brilliant over-acting Lesley Ann Warren Best d...,0
3,easily underrated film inn Brooks cannon Sure ...,0
4,typical Mel Brooks film much le slapstick movi...,0
5,n't comedic Robin Williams quirky/insane Robin...,0
6,art successfully make slow paced thriller. < b...,0
7,critically acclaimed psychological thriller ba...,0
8,NIGHT LISTENER Robin Williams Toni Collette Bo...,0
9,know Robin Williams God bless constantly shoot...,0


In [232]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
count_vec = CountVectorizer()
# cross validation
x_train, x_test, y_train, y_test = train_test_split(data.content, data.label, test_size=0.75, random_state=23)
# word count by CountVectorizer
x_train_mnb = count_vec.fit_transform(x_train)
#print(x_train_mnb[0])  
#print("happy")
x_test_mnb = count_vec.transform(x_test)
#print(x_test_mnb[0])

In [233]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
# calculate the tfidf of the text 
tfidf = TfidfTransformer()
x_train_tf = tfidf.fit_transform(x_train_mnb)
x_test_tf = tfidf.transform(x_test_mnb)
#print(x_train_tf)
#print("happy")
#print(x_test_tf)
#predict by native bayes    
mnb = MultinomialNB()
mnb.fit(x_train_tf, y_train)
print(classification_report(y_test, mnb.predict(x_test_tf)))

             precision    recall  f1-score   support

          0       0.86      0.83      0.85      9367
          1       0.84      0.87      0.85      9383

avg / total       0.85      0.85      0.85     18750



In [234]:
from sklearn import svm  
from sklearn.linear_model import SGDClassifier
#predict by SVM
#dtc = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)

model = svm.SVC(kernel='poly', degree=3, C=1.0)
model.fit(x_train_mnb, y_train)
print(classification_report(y_test, dtc.predict(x_test_mnb)))

             precision    recall  f1-score   support

          0       0.89      0.70      0.79      9367
          1       0.76      0.91      0.83      9383

avg / total       0.82      0.81      0.81     18750



In [236]:
from sklearn.ensemble import GradientBoostingClassifier
gbdt=GradientBoostingClassifier(n_estimators=200) 

#predict by GBDT
gbdt.fit(x_train_tf, y_train)
print(classification_report(y_test, gbdt.predict(x_test_tf.toarray())))

             precision    recall  f1-score   support

          0       0.80      0.86      0.83      9367
          1       0.85      0.79      0.82      9383

avg / total       0.83      0.83      0.83     18750



In [240]:
# the perfomance of SVM is best, let's start predict
count_vect = CountVectorizer()
tf_idf = TfidfTransformer()
X = data.content
Y = data.label
x_ct = count_vect.fit_transform(X)
x_tf = tf_idf.fit_transform(x_ct)
#svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
#svm.fit(x_tf, Y)
mnb_pre = MultinomialNB()
mnb_pre.fit(x_tf, Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [249]:
x_predict_ct = count_vect.transform(x_predict)
x_predict_tf = tf_idf.transform(x_predict_ct)
predict = mnb_pre.predict(x_predict_tf)

In [250]:
predict_data['sentiment'] = predict
predict_data.loc[predict_data.sentiment == 0, 'sentiment'] = 'negative'
predict_data.loc[predict_data.sentiment == 1, 'sentiment'] = 'positive'

In [251]:
predict_data.head()

,file_name,content,sentiment
0,./Desktop/sentiment/movie/0002.html,[Editor's note: Sites running 2.10 netnews wil...,positive
1,./Desktop/sentiment/movie/0003.html,"Set in the offices of two psychoanalysts,...",positive
2,./Desktop/sentiment/movie/0004.html,"Starring: Mickey Rourke, Robert DeNiro, Lisa B...",negative
3,./Desktop/sentiment/movie/0005.html,Harry Angel (played by Mickey Rourke) is ...,negative
4,./Desktop/sentiment/movie/0006.html,I've been a little lax in getting out mov...,positive


In [253]:
predict_data.to_csv('./Desktop/sentiment/result.csv',index=True)